### 3 D Human Modeling - Group 8 - Iteration_ 1 - March 7 2023

Requriements analysis:
<ul>
<li>
Node js

</li>
<li>
Media pipe library
</li>
<li>
open cv
</li>
<li>
Tensor Flow
</li>
<li>
Flask
</li>
</ul>


Code:

In [4]:
import cv2
import mediapipe as mp
import numpy as np

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [6]:


# Define the input image files
IMAGE_FILES = []

# Define the background color
BG_COLOR = (192, 192, 192) # gray

# Create a Pose object
with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5) as pose:

    # Iterate through each image file
    for idx, file in enumerate(IMAGE_FILES):
        # Read the image file
        image = cv2.imread(file)
        # Get the image height, width and number of channels
        image_height, image_width, _ = image.shape
        # Convert the BGR image to RGB before processing.
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Process the image using the Pose object
        results = pose.process(image_rgb)

        # Check if there are pose landmarks in the results
        if not results.pose_landmarks:
            continue
        
        # Print the coordinates of the nose landmark
        nose_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image_width
        nose_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * image_height
        print(f'Nose coordinates: ({nose_x}, {nose_y})')

        # Create a copy of the image for annotations
        annotated_image = image.copy()

        # Draw segmentation on the image.
        # To improve segmentation around boundaries, consider applying a joint
        # bilateral filter to "results.segmentation_mask" with "image".
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR
        annotated_image = np.where(condition, annotated_image, bg_image)

        # Draw pose landmarks on the image.
        mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        # Save the annotated image
        cv2.imwrite(f'/tmp/annotated_image{idx}.png', annotated_image)

        # Plot pose world landmarks.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)


Capturing video from the default camera:

In [7]:
cap = cv2.VideoCapture(0)

Initializing the MediaPipe Pose model:

In [11]:
# Starting the camera feed:
while cap.isOpened():
    # Reading a frame from the camera:
    success, image = cap.read()
    
    if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        continue

    # Converting the frame to RGB:
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Running the pose estimation model on the frame:
    results = pose.process(image)

    # Drawing the pose landmarks on the frame:
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    # Displaying the resulting frame with the pose landmarks:
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    
    # Checking if the 'Esc' key has been pressed to stop the loop:
    if cv2.waitKey(5) & 0xFF == 27:
        break


In [10]:
cap.release()